# Reviews

In [15]:
import pandas as pd
from funciones import *

In [16]:
file_path_user_reviews = '../data/raw/australian_user_reviews.json'

In [17]:
df_reviews = process_file(file_path_user_reviews, '../data/interim/csv/reviews.csv', load_and_normalize_user_reviews)

In [18]:
df_rev_cp = df_reviews.copy()
df_rev_cp.drop('user_url', axis=1, inplace=True)

In [19]:
# Convertir user_id a string
df_rev_cp['user_id'] = df_rev_cp['user_id'].astype(str)

# Extraer el número de la columna funny
df_rev_cp['funny'] = df_rev_cp['funny'].str.extract('(\d+)').fillna(0).astype(int)

# Extraer el año de la columna posted y convertir a Int64
df_rev_cp['posted'] = df_rev_cp['posted'].str.extract('(\d{4})').astype('Int64')

# Hacer lo mismo para la columna last_edited
df_rev_cp['last_edited'] = df_rev_cp['last_edited'].str.extract('(\d{4})').astype('Int64')

# Convertir item_id a int
df_rev_cp['item_id'] = df_rev_cp['item_id'].astype(int)


In [20]:
def extract_percentage(value):
    start = value.find('(')
    end = value.find('%')
    if start != -1 and end != -1:
        percentage = value[start+1:end]
        return int(percentage)
    return 0
df_rev_cp['helpful'] = df_rev_cp['helpful'].apply(extract_percentage)


In [21]:
df_reviews = df_rev_cp.copy()

In [22]:
df_reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,0,2011,<NA>,1250,0,True,Simple yet with great replayability. In my opi...
1,76561197970982479,0,2011,<NA>,22200,0,True,It's unique and worth a playthrough.
2,76561197970982479,0,2011,<NA>,43110,0,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,0,2014,<NA>,251610,75,True,I know what you think when you see this title ...
4,js41637,0,2013,<NA>,227300,0,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59300,76561198312638244,0,<NA>,<NA>,70,0,True,a must have classic from steam definitely wort...
59301,76561198312638244,0,<NA>,<NA>,362890,0,True,this game is a perfect remake of the original ...
59302,LydiaMorley,1,<NA>,<NA>,273110,50,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,0,<NA>,<NA>,730,0,True,:D


In [23]:
from textblob import TextBlob

def analyze_sentiment(text):
    if pd.isnull(text):
        return 1  # Neutral si no hay reseña
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0.05:
        return 2  # Positivo
    elif analysis.sentiment.polarity < -0.05:
        return 0  # Negativo
    else:
        return 1  # Neutral

# Aplicar la función de análisis de sentimientos a la columna review
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)

# Eliminar la columna review
df_reviews.drop('review', axis=1, inplace=True)

In [24]:
# decidi eliminar la columna por ser practicamente toda nula. 
df_reviews.drop(columns=['last_edited'], inplace=True)

In [25]:
df_reviews.to_csv('../data/interim/csv/final_reviews.csv', index=False)
df_reviews.to_parquet('../data/interim/parquet/final_reviews.parquet')